# 考察

極端に上がりの速さが遅いクラスターの分析を行う．

## 環境構築

In [1]:
# Notebook初期設定
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [3]:
# plotly関連
!pip install plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [4]:
DIR_IN = '../data/scatter_preprocess'
DIR_OUT = '../data/scatter_discussion'

In [5]:
FN_ALL = 'all_res.csv'

## 事前分析

In [20]:
df_all = pd.read_csv(os.path.join(DIR_IN, FN_ALL))

In [49]:
# speed_3fのレースごとの最大値を抜き出す
df_max_speed_3f = df_all.groupby('race_id')['speed_3f'].max().reset_index()
# speed_3fの最大値が50km/h以下のrace_idを抽出
raceids_slow_3f = \
    set(df_max_speed_3f[df_max_speed_3f['speed_3f'] < 50]['race_id'].unique())

In [50]:
df_tmp = df_all[df_all['race_id'].isin(raceids_slow_3f)].reset_index(drop=True)
df_tmp = df_tmp.drop_duplicates('race_id', ignore_index=True)

In [51]:
df_tmp['title'].unique()

array(['優駿牝馬', '天皇賞(春)', '菊花賞', 'エリザベス女王杯', '神戸新聞杯', '宝塚記念', '毎日杯',
       '阪神大賞典', 'ペガサスステークス', 'NHK杯', 'きさらぎ賞', '東京優駿', '毎日放送京都4歳特別',
       '読売マイラーズカップ', 'スポニチ賞金杯', '桜花賞', 'サファイヤステークス', 'デイリー杯3歳S', '新潟記念',
       '安田記念', '目黒記念', '鳴尾記念', 'アーリントンカップ', 'ラジオたんぱ杯3歳S', 'セントライト記念',
       '日経新春杯', 'サンスポ4歳牝馬特別', '京都新聞杯', 'マイルチャンピオンS', '報知杯4歳牝馬特別', '阪急杯',
       '京阪杯', 'NZT4歳S', '日刊スポシンザン記念', 'フェブラリーハンデ', '京王杯スプリングC',
       'アルゼンチン共和国杯', 'ダイヤモンドS', 'サンスポ阪神牝馬特別', '産經大阪杯', 'ラジオたんぱ3歳牝S',
       '報知杯弥生賞', '関屋記念', '皐月賞', '根岸ステークス', '京都大賞典', 'フジTVスプリングS',
       'フラワーカップ', '京都記念', '阪神3歳牝馬S', 'エプソムカップ', '朝日チャレンジカップ',
       'セントウルステークス', 'デイリー杯クイーンC', '新潟大賞典', 'サンケイ大阪杯', '東京新聞杯',
       '阪神3歳ステークス', '関西TVローズS', '共同通信杯4歳S', 'サンケイオールカマー', '七夕賞',
       'ジャパンカップ', '京成杯3歳ステークス', '牝馬東京タイムズ杯', 'クリスタルカップ', '京王杯オータムハンデ',
       '京都牝馬特別', 'スワンステークス', '毎日王冠', 'スプリンターズS', '天皇賞(秋)', '中山牝馬ステークス',
       'クイーンステークス', '中山記念', '日経賞'], dtype=object)

In [52]:
df_tmp['place'].unique()

array(['東京', '京都', '阪神', '新潟'], dtype=object)

In [53]:
pd.to_datetime(df_tmp['date']).dt.year.value_counts()

1988    70
1989    59
1987    54
1986    51
1990    38
1992    37
1991    29
1993    13
Name: date, dtype: int64